# core

> Read and query chronicle parquet files.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import polars as pl
import pyarrow.parquet as pq
import pyarrow.dataset as ds
from s3fs import S3FileSystem
import pandas as pd
import plotly.express as px
from fastcore.basics import patch

## Read chronicle parquet files

Chronicle collects and stores logs and metrics in a series of parquet files.

Use `read_chronicle()` to read either logs or metrics, by specifying the path to the parquet set you need.

The file tree looks like this, with `logs` and `metrics` in separate folders inside `v1`.

``` bash
.
└── v1/
    ├── logs/
    └── metrics/
```

Inside both `logs` and `metrics` the data is stored by date, separated by year, month and day.

``` bash
.
└── v1/
    ├── logs/
    │   └── 2023/
    │       ├── 02/
    │       │   ├── 01
    │       │   ├── 02
    │       │   ├── 03
    │       │   ├── 04
    │       │   ├── 05
    │       │   └── ...
    │       ├── 03
    │       ├── 04
    │       └── ...
    └── metrics/
        └── 2023/
            ├── 02/
            │   ├── 01
            │   ├── 02
            │   ├── 03
            │   ├── 04
            │   ├── 05
            │   └── ...
            ├── 03
            ├── 04
            └── ...
```

In [ ]:
#| export
def read_chronicle(
        path: str # Path to dataset
    ) -> pl.DataFrame:
    "Read a chronicle parquet file into a polars dataframe."
    return pl.from_arrow(pq.read_table(path))


In [ ]:
z = read_chronicle("./data/v1/metrics")
assert type(z) == pl.dataframe.frame.DataFrame

z = read_chronicle("./data/v1/logs")
assert type(z) == pl.dataframe.frame.DataFrame

## Analyse metrics

In [ ]:
#| export
@pl.api.register_dataframe_namespace("metrics")
class ChronicleMetrics:
    def __init__(self, 
                 df: pl.DataFrame # A `polars` DataFrame
                 ) -> pl.DataFrame:
        "Initialise a chronicle metrics class"
        self._df = df



In [ ]:
#| export
@patch
def describe(self: ChronicleMetrics) -> pl.DataFrame:
    "Reads metrics dataframe and returns a pandas dataframe with summary of service, name and description of all metrics"
    return (
        self._df
        .groupby("service", "name")
        .agg(
            pl.col("description").unique(),
            pl.col("value_column").unique(),
        )
        .with_columns(
            pl.col("description").arr.join(", "),
            pl.col("value_column").arr.join("")
        )
        .sort("service", "name")
        .to_pandas()
    )

The metrics data has a single row for each collected metric.

Use `describe()` to get a DataFrame of the unique metrics in the metrics data, containing the `service`, `name` and `description` of each metric.

In [ ]:

m = read_chronicle("./data/v1/metrics/").metrics.describe()
assert list(m) == ['service', 'name', 'description', 'value_column']
m

In [ ]:
#| export
@patch
def filter(self: ChronicleMetrics, 
        name:str, # name of metric to extract
        alias:str = None # alias to use for new column
    ) -> pd.DataFrame:
    "Extract a single metric from a metrics dataframe"
    if alias == None:
        alias = name
    return (
        self._df
        .lazy()
        .filter(pl.col("name") == name)
        .sort(pl.col("host"), pl.col("timestamp"))
        .select([
            "host",
            pl.col("timestamp"),
            pl.col("value_float").alias(alias)
        ])
        .collect()
        .to_pandas()
    )


The `name` argument is used to filter the DataFrame on the `name` column.

In [ ]:
m = read_chronicle("./data/v1/metrics/").metrics.filter("rsconnect_system_memory_used")
assert type(m) == pd.DataFrame
assert list(m) == ['host', 'timestamp', 'rsconnect_system_memory_used']

m = read_chronicle("./data/v1/metrics/").metrics.filter("rsconnect_system_memory_used", "memory")
assert type(m) == pd.DataFrame
assert list(m) == ['host', 'timestamp', 'memory']

m


In [ ]:
#| export
@patch
def plot(
        self:ChronicleMetrics, # metrics dataframe
        name:str, # name of metric to extract
        alias:str = None # alias to use for new column
    ) -> px.line: 
    "Plot a selected metric using a Plotly line plot"

    dat = self._df.metrics.filter(name, alias) 
    fig = px.line(dat, x='timestamp', y=alias, line_group="host", color="host")
    return fig

In [ ]:
m = read_chronicle("./data/v1/metrics/")
p = m.metrics.plot("rsconnect_system_memory_used", "memory")
assert str(type(p)) == "<class 'plotly.graph_objs._figure.Figure'>"


read_chronicle("./data/v1/metrics/").metrics.plot("rsconnect_system_memory_used", "memory")

## Analyse logs

In [ ]:

#| export
@pl.api.register_dataframe_namespace("logs")
class ChronicleLogs:
    def __init__(self, 
                 df: pl.DataFrame # A polars data frame
                 ) -> pl.DataFrame:
        "Initialise a chronicle logs DataFrame"
        self._df = df

#### Filter logs on type

You can 

In [ ]:
#| export
@patch
def filter_type(self: ChronicleLogs,
                value: str # Value to extract 
    ) -> pd.DataFrame:
    "Extract all logs where type == value"
    return (
        self._df
        .lazy()
        .with_columns([
            (pl.col("body").str.json_path_match(f"$.{value}").alias(f".{value}")),
            (pl.col("body").str.json_path_match("$.type").alias(".type"))
        ])
        .filter(pl.col(f".{value}").is_not_null())
        .select(["service", "host", "timestamp", f".{value}", ".type", "body"])
        .sort("service", "host", f".{value}", "timestamp")
        .collect()
    )

In [ ]:
logs = read_chronicle("./data/v1/logs").logs.filter_type("username")
assert type(logs) == pl.DataFrame

# assert logs


In [ ]:

read_chronicle("./data/v1/logs").logs.filter_type("username")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()